In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [5]:
df = fetch_hourly_rides(12)

2025-03-06 03:14:48,227 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:14:48,244 INFO: Initializing external client
2025-03-06 03:14:48,244 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:14:48,858 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.26s) 


In [6]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-06 03:00:00+00:00,159,0
1,2025-03-05 20:00:00+00:00,35,0
2,2025-03-06 03:00:00+00:00,162,8
3,2025-03-05 22:00:00+00:00,69,0
4,2025-03-06 02:00:00+00:00,145,0
...,...,...,...
4027,2025-03-05 23:00:00+00:00,112,1
4028,2025-03-05 20:00:00+00:00,261,57
4029,2025-03-05 17:00:00+00:00,231,137
4030,2025-03-05 16:00:00+00:00,227,0


In [7]:
df_pred = fetch_predictions(12)

2025-03-06 03:14:58,107 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:14:58,107 INFO: Initializing external client
2025-03-06 03:14:58,107 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:14:58,807 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.85s) 


In [8]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,212,0.0,2025-03-06 08:00:00+00:00
1,52,0.0,2025-03-06 08:00:00+00:00
2,129,1.0,2025-03-06 08:00:00+00:00
3,145,-0.0,2025-03-06 08:00:00+00:00
4,140,51.0,2025-03-06 08:00:00+00:00
...,...,...,...
247,59,0.0,2025-03-06 08:00:00+00:00
248,200,0.0,2025-03-06 08:00:00+00:00
249,47,0.0,2025-03-06 08:00:00+00:00
250,187,0.0,2025-03-06 08:00:00+00:00


In [9]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [10]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand


In [11]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [12]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [13]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [14]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [15]:
mae_by_hour["MAE"].mean()

nan